<h2><b>Importing libraries</b></h2>

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import mplfinance as mpf
from pandas_datareader import data
import matplotlib.dates as mdates
import seaborn as sns
import yfinance as yf
import tensorflow as tf
from tensorflow import keras
import pandas_datareader as web
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

/opt/anaconda3/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


<h2><b>Getting data</b></h2>

In [2]:
histVale = web.DataReader('VALE', data_source='yahoo', start="2018-01-01", end="2018-12-30")
histAzul = web.DataReader('AZUL4.SA', data_source='yahoo', start="2018-01-01", end="2018-12-30")
histBB = web.DataReader('BBAS3.SA', data_source='yahoo', start="2018-01-01", end="2018-12-30")
histPetro = web.DataReader('PBR', data_source='yahoo', start="2018-01-01", end="2018-12-30")
histItau = web.DataReader('ITUB', data_source='yahoo', start="2018-01-01", end="2018-12-30")

In [3]:
historicStock = [histVale, histAzul, histBB, histPetro, histItau]

In [4]:
monthlyInflation2018 = [0.0076, 0.0007, 0.0064, 0.0057, 0.0138, 0.0187, 0.0051, 0.0070, 0.0152, 0.0089, -0.0049, -0.0108]

<h2><b>Calculating daily inflation</b></h2>

In [5]:
dayMonth30 = [4,6,9,11]
dayMonth31 = [1,3,5,7,8,10,12]
        
def calculate_daily_inflation(monthlyInflation):
    dailyInflation = []
    inflationHigh = []
    inflationLow = []
    inflationOpen = []
    inflationClose = []
    
    for index, inf in enumerate(monthlyInflation):
        if np.isin(index, dayMonth30):
            dailyInflation.append((1 + inf) ** (1/float(30)))
        elif np.isin(index, dayMonth31):
            dailyInflation.append((1 + inf) ** (1/float(31)))
        else:
            dailyInflation.append((1 + inf) ** (1/float(28)))
    
    for stock in historicStock:
        stock["Month"] = stock.index.month
        total = 1
        inflation = []
        for index, row in stock.iterrows():
            month = int(row['Month'])
            if dailyInflation[month - 1] >= 1:
                total = total * dailyInflation[month - 1]
            else:
                total = total / dailyInflation[month - 1]
            inflation.append(total)
        inflationReversed = np.flip(inflation)
        stock["DailyInflation"] = inflationReversed
        
    for stock in historicStock:
        for index, row in stock.iterrows():
            inflationHigh = round(stock["High"] * stock["DailyInflation"], 2)
            inflationLow = round(stock["Low"] * stock["DailyInflation"], 2)
            inflationOpen = round(stock["Open"] * stock["DailyInflation"], 2)
            inflationClose = round(stock["Close"] * stock["DailyInflation"], 2) 
        stock["InflationHigh"] = inflationHigh
        stock["InflationLow"] = inflationLow
        stock["InflationOpen"] = inflationOpen
        stock["InflationClose"] = inflationClose

In [6]:
calculate_daily_inflation(monthlyInflation2018)

<h2><b>Calculating short and long EMAs, MACD and Signal lines</b></h2>

In [7]:
def calculate_lines():
    for stock in historicStock:
        stock["ShortEMA"] = stock["Close"].ewm(span = 12, adjust = False).mean()
        stock["LongEMA"] = stock["Close"].ewm(span = 26, adjust = False).mean()
        stock["MACD"] = stock["ShortEMA"] - stock["LongEMA"]
        stock["Signal"] = stock["MACD"].ewm(span = 9, adjust = False).mean()
        stock["InflationShortEMA"] = stock["InflationClose"].ewm(span = 12, adjust = False).mean()
        stock["InflationLongEMA"] = stock["InflationClose"].ewm(span = 26, adjust = False).mean()
        stock["InflationMACD"] = stock["InflationShortEMA"] - stock["InflationLongEMA"]
        stock["InflationSignal"] = stock["InflationMACD"].ewm(span = 9, adjust = False).mean()

In [8]:
calculate_lines()

<h2><b>Calculating Relative Strength Index (RSI)</b></h2>

In [9]:
def calculate_RSI(window):
    for stock in historicStock:
        delta = stock["Close"].diff()
        up_days = delta.copy()
        up_days[delta<=0]=0.0
        down_days = abs(delta.copy())
        down_days[delta>0]=0.0
        RS_up = up_days.rolling(window).mean()
        RS_down = down_days.rolling(window).mean()
        rsi= 100-100/(1+RS_up/RS_down)
        stock["RSI"] = rsi
        
        delta = stock["InflationClose"].diff()
        up_days = delta.copy()
        up_days[delta<=0]=0.0
        down_days = abs(delta.copy())
        down_days[delta>0]=0.0
        RS_up = up_days.rolling(window).mean()
        RS_down = down_days.rolling(window).mean()
        rsi= 100-100/(1+RS_up/RS_down)
        stock["InflationRSI"] = rsi

In [10]:
calculate_RSI(14)

<h2><b>Function to plot RSI</b></h2>

In [11]:
def plot_rsi_close(stock, stockName, inflation=False):
    scaler = MinMaxScaler()
    stockScaled = pd.DataFrame(scaler.fit_transform(stock), columns=stock.columns)

    plt.figure(figsize = (20, 10))
    if inflation == False:
        ax = stockScaled["RSI"].plot()
        stockScaled["Close"].plot()
    else:
        ax = stockScaled["InflationRSI"].plot()
        stockScaled["InflationClose"].plot()
    ax.axhline(y = 0.7, color = 'red')
    ax.axhline(y = 0.3, color = 'lime')
    plt.title(f'Relação RSI e Close da {stockName}')
    plt.legend()
    plt.show()

<h2><b>Calculating Money Flow Index (MFI)</b></h2>

In [12]:
def calculate_MFI(window):
    for stock in historicStock:
        stock["TypicalPrice"] = (stock["High"] + stock["Low"] + stock["Close"]) / 3
        delta = stock["TypicalPrice"].diff()
        up_days = delta.copy()
        up_days[delta<=0]=0.0
        up_days[delta>0]=stock["TypicalPrice"]
        down_days = abs(delta.copy())
        down_days[delta>0]=0.0
        down_days[delta<=0]=stock["TypicalPrice"]
        stock["1DayPositiveMoneyFlow"] = up_days * stock["Volume"]
        stock["1DayNegativeMoneyFlow"] = down_days * stock["Volume"]
        stock["nDayPositiveMoneyFlow"] = stock["1DayPositiveMoneyFlow"].rolling(window).sum()
        stock["nDayNegativeMoneyFlow"] = stock["1DayNegativeMoneyFlow"].rolling(window).sum()
        stock["MoneyFlowRatio"] = stock["nDayPositiveMoneyFlow"] / stock["nDayNegativeMoneyFlow"]
        stock["MoneyFlowIndex"] = 100 - (100 / (1 + stock["MoneyFlowRatio"]))

        stock["InflationTypicalPrice"] = (stock["InflationHigh"] + stock["InflationLow"] + stock["InflationClose"]) / 3
        delta = stock["InflationTypicalPrice"].diff()
        up_days = delta.copy()
        up_days[delta<=0]=0.0
        up_days[delta>0]=stock["InflationTypicalPrice"]
        down_days = abs(delta.copy())
        down_days[delta>0]=0.0
        down_days[delta<=0]=stock["InflationTypicalPrice"]
        stock["Inflation1DayPositiveMoneyFlow"] = up_days * stock["Volume"]
        stock["Inflation1DayNegativeMoneyFlow"] = down_days * stock["Volume"]
        stock["InflationnDayPositiveMoneyFlow"] = stock["Inflation1DayPositiveMoneyFlow"].rolling(window).sum()
        stock["InflationnDayNegativeMoneyFlow"] = stock["Inflation1DayNegativeMoneyFlow"].rolling(window).sum()
        stock["InflationMoneyFlowRatio"] = stock["InflationnDayPositiveMoneyFlow"] / stock["InflationnDayNegativeMoneyFlow"]
        stock["InflationMoneyFlowIndex"] = 100 - (100 / (1 + stock["InflationMoneyFlowRatio"]))

In [13]:
calculate_MFI(14)

<h2><b>Function to plot MFI</b></h2>

In [14]:
def plot_mfi(stock, stockName, inflation=False):

    plt.figure(figsize = (20, 10))
    if inflation == False:
        ax = stock["MoneyFlowIndex"].plot()
    else:
        ax = stock["InflationMoneyFlowIndex"].plot()
    ax.axhline(y = 20, color = 'red')
    ax.axhline(y = 80, color = 'lime')
    plt.title(f'MFI da {stockName}')
    plt.legend()
    plt.show()

<h2><b>Calculating Stochastic Oscillator</b></h2>

In [15]:
def calculate_stochastic_oscillator(window):
    for stock in historicStock:
        nHigh = stock["High"].rolling(window).max()
        nLow = stock["Low"].rolling(window).min()
        stock["StochasticOscillator"] = ((stock["Close"] - nLow) / (nHigh - nLow)) * 100
        
        nHigh = stock["InflationHigh"].rolling(window).max()
        nLow = stock["InflationLow"].rolling(window).min()
        stock["InflationStochasticOscillator"] = ((stock["InflationClose"] - nLow) / (nHigh - nLow)) * 100

In [16]:
calculate_stochastic_oscillator(14)

<h2><b>Function to plot Stochastic Oscillator</b></h2>

In [17]:
def plot_stochastic_oscillator(stock, stockName, inflation=False):

    plt.figure(figsize = (20, 10))
    if inflation == False:
        ax = stock["StochasticOscillator"].plot()
    else:
        ax = stock["InflationStochasticOscillator"].plot()
    ax.axhline(y = 20, color = 'red')
    ax.axhline(y = 80, color = 'lime')
    plt.title(f'Oscilador estocástico da {stockName}')
    plt.legend()
    plt.show()

<h2><b>Function to plot correlation matrix</b></h2>

In [18]:
def show_corr_matrix(stock, inflation=False):

    if inflation == False:
        corrMatrix = stock[["Close", "RSI", "MoneyFlowIndex", "StochasticOscillator", "ShortEMA", "LongEMA", "MACD"]].corr()
    else:
        corrMatrix = stock[["InflationClose", "InflationRSI", "InflationMoneyFlowIndex", "InflationStochasticOscillator", "InflationShortEMA", "InflationLongEMA", "InflationMACD"]].corr()

    sns.heatmap(corrMatrix, annot=True)
    plt.figure(figsize = (100,100))
    plt.show()

<h2><b>Function to predict traditional SVM</b></h2>

In [19]:
svmRegressor = svm.SVR(kernel="linear")
svmRegressorNu = svm.NuSVR(kernel="linear")

In [20]:
def predict_svm_traditional(stock, inflation=False):
    if inflation == False:
        stockX = stock[["RSI", "MoneyFlowIndex", "StochasticOscillator", "ShortEMA", "MACD"]][15:]
        stockY = stock["Close"][15:]
    else:
        stockX = stock[["InflationRSI", "InflationMoneyFlowIndex", "InflationStochasticOscillator", "InflationShortEMA", "InflationMACD"]][15:]
        stockY = stock["InflationClose"][15:]
    X = np.array(stockX)
    y = np.array(stockY)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, shuffle=False)

    svmRegressor.fit(X_train, y_train)
    
    y_pred = svmRegressor.predict(X_test)
    print('Predicted values: ', y_pred)
    print('Testing values: ', y_test)
    print('Coefficients: \n', svmRegressor.coef_)
    
    return y_pred, y_test, y_train

In [21]:
def predict_plot_svm(stock, inflation=False):
    y_pred, y_test, y_train = predict_svm_traditional(stock, inflation)
    
    print('R^2 value: ', r2_score(y_test, y_pred))
    print('MAE score: ', mean_absolute_error(y_test, y_pred))
    print('RMSE value: ', mean_squared_error(y_test, y_pred, squared=False))
    
    plt.plot(y_test, label="Real value")
    plt.plot(y_pred, label="Prediction")
    plt.legend()
    plt.show()

<h2><b>Function to predict Nu SVM</b></h2>

In [22]:
def predict_svm_nu(stock, inflation=False):
    if inflation == False:
        stockX = stock[["RSI", "MoneyFlowIndex", "StochasticOscillator", "ShortEMA", "MACD"]][15:]
        stockY = stock["Close"][15:]
    else:
        stockX = stock[["InflationRSI", "InflationMoneyFlowIndex", "InflationStochasticOscillator", "InflationShortEMA", "InflationMACD"]][15:]
        stockY = stock["InflationClose"][15:]
    X = np.array(stockX)
    y = np.array(stockY)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, shuffle=False)

    svmRegressorNu.fit(X_train, y_train)
    
    y_pred = svmRegressorNu.predict(X_test)
    print('Predicted values: ', y_pred)
    print('Testing values: ', y_test)
    print('Coefficients: \n', svmRegressorNu.coef_)
    
    return y_pred, y_test, y_train

In [23]:
def predict_plot_svm_nu(stock, inflation=False):
    y_pred, y_test, y_train = predict_svm_nu(stock, inflation)
    
    print('R^2 value: ', r2_score(y_test, y_pred))
    print('MAE score: ', mean_absolute_error(y_test, y_pred))
    print('RMSE value: ', mean_squared_error(y_test, y_pred, squared=False))
    
    plt.plot(y_test, label="Real value")
    plt.plot(y_pred, label="Prediction")
    plt.legend()
    plt.show()

In [25]:
scaler = MinMaxScaler()
stockScaled = []
stocksScaled = []

for stock in historicStock:
    stocksScaled.append(stock)
    
for stock in stocksScaled:
    stockScaled.append(pd.DataFrame(scaler.fit_transform(stock), columns=historicStock[0].columns))

In [34]:
stockScaled[1]

,High,Low,Open,Close,Volume,Adj Close,Month,DailyInflation,InflationHigh,InflationLow,...,MoneyFlowIndex,InflationTypicalPrice,Inflation1DayPositiveMoneyFlow,Inflation1DayNegativeMoneyFlow,InflationnDayPositiveMoneyFlow,InflationnDayNegativeMoneyFlow,InflationMoneyFlowRatio,InflationMoneyFlowIndex,StochasticOscillator,InflationStochasticOscillator
0,0.349730,0.369146,0.354458,0.368018,0.095326,0.368018,0.0,1.000000,0.391261,0.408928,...,NaN,0.396611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.344324,0.354270,0.354458,0.363089,0.042827,0.363089,0.0,0.994709,0.385234,0.393535,...,NaN,0.387539,0.000000,0.061007,NaN,NaN,NaN,NaN,NaN,NaN
2,0.355135,0.366391,0.345865,0.372399,0.130079,0.372399,0.0,0.989420,0.395279,0.405336,...,NaN,0.397980,0.130256,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
3,0.362162,0.353168,0.371106,0.370756,0.099849,0.370756,0.0,0.984132,0.401808,0.391483,...,NaN,0.394728,0.000000,0.142923,NaN,NaN,NaN,NaN,NaN,NaN
4,0.340541,0.361983,0.365199,0.354874,0.032826,0.354874,0.0,0.978847,0.379709,0.399692,...,NaN,0.384457,0.000000,0.046665,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,0.697297,0.661708,0.661654,0.715772,0.384587,0.715772,1.0,0.014740,0.615269,0.581837,...,0.477976,0.605957,0.438593,0.000000,0.357411,0.480584,0.058363,0.477667,0.643326,0.622030
241,0.762162,0.723416,0.714286,0.797919,0.389566,0.797919,1.0,0.011054,0.675038,0.639302,...,0.610096,0.671003,0.461214,0.000000,0.442541,0.365475,0.095790,0.609841,1.000000,1.000000
242,0.798378,0.752617,0.778733,0.791895,0.244294,0.791895,1.0,0.007368,0.708187,0.665983,...,0.696868,0.689148,0.292188,0.000000,0.496473,0.290490,0.134702,0.696682,0.847358,0.844930
243,0.762703,0.755372,0.778195,0.792990,0.199056,0.792990,1.0,0.003684,0.674535,0.668035,...,0.630307,0.678535,0.000000,0.339040,0.450166,0.344301,0.103523,0.630153,0.851272,0.846918
